In [1]:
from tables import *
from datetime import datetime, timedelta

class TLE(IsDescription):
    epoch     = Float64Col(pos=0)
    norad_id  = Int64Col(pos=1)
    line_one  = StringCol(80, pos=2)
    line_two  = StringCol(80, pos=3)

In [3]:
h5file = open_file('./data/Indexed_TLE/reduced.h5', mode="r")

In [7]:
datetime.utcfromtimestamp(h5file.root.tle_sorted.cols.epoch[-1])

datetime.datetime(2017, 2, 14, 23, 59, 51, 786804)

In [ ]:
h5file.root.readout.cols.norad_id.create_index()

In [15]:
# Prune and sort to match the AIS data that we care about
stripped = h5file.create_table(h5file.root, 'tle_boxed', TLE, "TLE's stripped down to useful date ranges")

In [21]:
# AIS Valid date ranges are:
#2009 2008-12-31 23:58:59 2009-02-01 00:00:00
#2010 2009-12-29 17:05:00 2010-02-01 01:11:00
#2011 2010-12-31 23:58:59 2011-01-31 23:58:59
#2012 2012-01-04 15:59:48 2012-02-01 00:00:03
#2013 2012-12-31 23:59:58 2013-02-01 00:00:04
#2014 2013-12-31 23:57:43 2014-02-01 00:00:05
#2015 2015-01-01 00:00:02 2015-01-31 23:59:59
#2016 2016-01-01 00:00:01 2016-01-31 23:59:59
#2017 2017-01-01 00:00:00 2017-01-31 23:59:58

tle_extra = timedelta(weeks=2)
# We are going to assume we only need TLE's from Dec15 to Feb15
for year in range(2009, 2018):
    start_dt = datetime(year, 1, 1) - tle_extra
    end_dt = datetime(year, 2, 1) + tle_extra
    condition = "(epoch>={}) & (epoch<={})".format(start_dt.timestamp(), end_dt.timestamp())
    appended = h5file.root.tle.append_where(stripped, condition=condition)
    print("Year %i: records: %i" % (year, appended))


Year 2009: records: 790839
Year 2010: records: 860096
Year 2011: records: 866673
Year 2012: records: 896767
Year 2013: records: 498829
Year 2014: records: 517964
Year 2015: records: 535551
Year 2016: records: 1079155
Year 2017: records: 1049756


In [22]:
stripped.flush()

In [ ]:
# Create a full index to make it possible to completely sort with copy operation

In [34]:
stripped.cols.epoch.create_index(kind="full")

7095630

In [35]:
# Create a copy that is CSI on epoch, and also indexed on id
stripped.copy(h5file.root, "tle_sorted", sortby="epoch")

/tle_sorted (Table(7095630,)) "TLE's stripped down to useful date ranges"
  description := {
  "epoch": Float64Col(shape=(), dflt=0.0, pos=0),
  "norad_id": Int64Col(shape=(), dflt=0, pos=1),
  "line_one": StringCol(itemsize=80, shape=(), dflt=b'', pos=2),
  "line_two": StringCol(itemsize=80, shape=(), dflt=b'', pos=3)}
  byteorder := 'little'
  chunkshape := (372,)

In [ ]:
h5file.root.tle_sorted.cols.epoch.create_index()

In [40]:
h5file.root.tle_sorted.cols.norad_id.create_index()

7095630

In [41]:
h5file.close()

In [46]:
h5file = open_file("/mnt/disk100/persist_home/meawoppl/reexport.h5", mode="r+")

In [ ]:
del h5file.root.tle

In [44]:
h5file.root.tle_boxed._f_remove()

In [48]:
h5file.close()